<h1 style="text-align: center;">Programa de Alocação de Voluntários</h1>
<p style="text-align: center; font-size:12px;">Leonardo Gama Assumpção - UFRJ - 30/03/2020</p>

Em tempos recentes, devido à pandemia, o Restaurante Universitário passou a oferecer as refeições a um público mais restrito, na forma de quentinhas entregues no Hospital Universitário, no Alojamento Estudantil e na Vila Residencial. Em particular para este último público, 200 a 250 quentinhas têm sido entregues duas vezes ao dia (para almoço e janta) na Associação de Moradores da Vila Residencial.

Com isso, estudantes universitários da Vila Residencial têm se oferecido para ajudar na distribuição das quentinhas que chegam a cada turno, sendo oportuno, portanto, um sistema que permita a alocação de voluntários para dividir nossos esforços de modo a facilitar a participação de todos segundo a disponibilidade de cada um, evitando possíveis sobrecargas.

No que segue, apresentamos a modelagem do problema como um programa de otimização linear com variáveis binárias.

In [1]:
import numpy as np
from sys import path
path.append("../src")
import schedule as sc

cp = sc.cp  # <= (import cvxpy as cp)
# from imp import reload ; reload(sc)

In [2]:
# Arquivos de Dados:
table_fname = sc.default_data_file
volunteers_fname = sc.default_volunteers_file
print('# DADOS:        "{}"\n# VOLUNTÁRIOS:  "{}"'.format(table_fname, volunteers_fname))

# DADOS:        "../data/tabela.txt"
# VOLUNTÁRIOS:  "../data/voluntarios.txt"


&nbsp;
## Dados
&nbsp;

In [3]:
names = sc.read_data(volunteers_fname).split("\n")
sc.print_enumeration(names)

01: Alex D. O.
02: Andria S. O. R.
03: Annatercia G. P.
04: Beatriz S. B. S.
05: Bianca B. D.
06: Breno Henrique O. S.
07: Bryan P.
08: Carlos Eduardo S.
09: Charlie V. S.
10: Daisy S. D. Z. A.
11: Danilo S. A. B.
12: Fabiana P. P. V.
13: Felipe O. M.
14: Fred F. M.
15: Gabriel P. C.
16: Hariom N. C.
17: Jonatã A. S.
18: Khaian L.
19: Leon L. G. A. S.
20: Leonardo G. A.
21: Letícia B. P. A. R.
22: Lorena A. D. B.
23: Lucas D. I. C.
24: Markus Vinicius G. H. C.
25: Matheus A. O.
26: Miguel A. P. R.
27: Otávio L. J.
28: Thiago C. P.
29: # Amanda A. C. L.
30: # Ferdinando R. F.
31: # Hélen B. S. M.
32: # Joyce A. N.


In [4]:
sc.print_enumeration(sc.read_data(table_fname).split("\n"))

01: 0	O	O	X	X	X	X	X	X	X	X	X	X	O	O
02: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
03: 0	X	X	X	X	O	O	X	X	O	O	X	X	X	X
04: 0	O	O	X	X	O	O	X	X	O	O	O	O	X	X
05: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
06: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
07: 0	X	X	X	X	O	O	X	X	O	O	X	X	O	O
08: 0	O	X	O	X	O	X	O	X	O	X	O	X	O	X
09: 0	O	O	O	O	X	X	X	X	X	X	X	X	O	O
10: 0	O	O	O	O	X	X	O	O	X	X	O	O	O	O
11: 0	O	O	X	X	O	O	O	O	X	X	O	O	O	O
12: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
13: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
14: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
15: 0	O	X	X	X	X	X	X	X	X	X	X	X	O	X
16: 0	X	X	O	O	O	O	O	O	X	X	OO	OO	OO	OO
17: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
18: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
19: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
20: 0	O	O	X	X	X	X	X	X	X	X	X	X	X	X
21: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
22: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
23: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
24: 0	.	.	.	.	.	.	.	.	.	.	.	.	.	.
25: 0	X	X	O	O	O	O	O	O	O	O	O	O	X	X
26: 0	O	O	.	.	O	O	.	.	O	O	.	.	O	O
27: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
28: 0	O	OO	O	OO	O	OO	O	OO	X	X	X	X	O	O
29: 0	O	O	O	O	O	O	O	O	O	O	O	O	O	O
30: 0	

# Modelo
A proposta do programa é oferecer uma alocação para os voluntários por um período fixo, de modo a evitar que pessoas fiquem sobrecarregadas e levar em conta as preferências de dias e horários de cada pessoa na tomada de decisão.

Tal problema pode ser modelado como um programa de otimização linear com variáveis binárias, como veremos a seguir. Comecemos fixando algumas

## Definições
- $M := \texttt{min-staff}$: número mínimo de voluntários por turno (atualmente, $M=7$)
- $\texttt{i}$: número (código) da pessoa. vai de $0$ a $N-1$ (atualmente, $N=27$)
- $\texttt{j}$: número (código) do turno. vai de $0$ a $T-1$ (atualmente, $T=14$)
- $A_{ij}$: indicadora ($0$ ou $1$) da pessoa $\texttt{i}$ estar disponível no turno $\texttt{j}$
- $P_{ij}$: indicadora ($0$ ou $1$) da pessoa $\texttt{i}$ ter preferência pelo turno $\texttt{j}$
- $w_i$: número (inteiro) de turnos contribuídos pela pessoa $\texttt{i}$ na semana anterior.

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**Variável de decisão do nosso problema:**
- $Z_{ij}$: indicadora da pessoa $\texttt{i}$ ser alocada para colaborar no turno $\texttt{j}$

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**Variável auxiliar** (para algumas contas ficarem mais compreensíveis)**:**
- $L_i = \texttt{load}\left[\texttt{i}\right]$: número de turnos alocados para a pessoa $\texttt{i}$.

Para ilustrar a definição, observemos que $L_i$ é exatamente a soma da linha $\texttt{i}$ de $Z$:
$$L_i = \sum_{j=0}^{T-1} Z_{ij}$$

## Função de Custo
Temos diversos objetivos concorrentes:
<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**1. Minimizar a carga máxima por pessoa:**

Sendo possível alocar $M$ pessoas por turno com cada uma das $N$ pessoas contribuindo com no máximo $4$ turnos, por exemplo, tal cenário será preferível a qualquer outro com uma ou mais pessoas sobrecarregadas com $5$ ou $6$ turnos na semana.
Uma expressão matemática para esse custo seria
$$\max_i \left| \sum_{j=0}^{T-1} Z_{ij} \right| = \max_i |L_i| = \max_i L_i = \left\lVert L \right\rVert_\infty.$$

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**2. Evitar disparidades no volume de trabalho:**

Resguardando-se que o sistema consiga alocar o mínimo de $M$ pessoas por turno, preferimos um cenário no qual todos contribuem de modo semelhante, em contraste a outro cenário onde alguns trabalham $4$ turnos e outros trabalham $2$.
Assim, se $\overline{L}$ é a média dos $L_i$, ajustamos uma constante $\beta$ de normalização e escrevemos:
$$\beta \sum_{i=0}^{N-1} \left| L_i - \overline{L} \right| = \beta \, \left\lVert L-\overline{L}\right\rVert_1$$

Assim, se $\mathbb{1} := (1, \ldots, 1)$, como $L = Z \cdot \mathbb{1}$, nossa função de custo seria dada, até agora, por
$$f(Z) = \varphi(L) = \left\lVert L \right\rVert_\infty + \beta \, \left\lVert L-\overline{L}\right\rVert_1.$$

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

**3. Favorecer cenários com contribuições em pares (almoço + janta):**

Tendo em vista a hipótese das transições entre os turnos de almoço e jantar serem mais simples (possivelmente com menor risco de imprevistos) ao envolver um número menor de pessoas, é razoável impor uma redução (dada por um parâmetro $\gamma$) na função de custo cada vez que uma mesma pessoa for alocada para colaborar nos dois turnos de um mesmo dia (ao invés de contribuir meio período, em dois dias distintos).

Modelar tal bônus (ou "penalização negativa") com funções indicadoras elevaria proibitivamente a complexidade do problema; então usamos o artifício de criar uma variável nova para cada par (dia, pessoa): uma matriz $D$ de dimensões $N \times 7$, definida da maneira que explicitamos a seguir.

Imagine que uma pessoa $i$ vai contribuir no almoço e no jantar de um mesmo dia. Ou seja, existiria um natural $j$ tal que ambos $Z_{i,j}$ e $Z_{i,j+1}$ valem $1$, sendo $j$ e $j+1$ correspondentes a almoço e janta de um mesmo dia $k$. Repare que $j$ tem de ser par, já que começamos a contar do zero; por exemplo, se $j=8$, teríamos $8$ e $9$ sendo almoço e jantar do dia $k := j\,/\,2 = 4$ (lembrando que os dias vão de $0$ a $6$).
Desse modo, se deixarmos $D_{i,k}$ assumir qualquer valor que satisfaça às restrições $D_{i,k} \le Z_{i,j}$ e $D_{i,k} \le Z_{i,j+1}$, ao minimizar $f(Z) - D_{i,k}$, devido ao sinal negativo, a variável $D_{i,k}$ atingiria otimalidade em seu maior valor possível, que é exatamente o mínimo entre $Z_{i,j}$ e $Z_{i,j+1}$. Mas lembrando que as entradas de $Z$ são binárias, isso é equivalente à implicação
$$Z_{i,j} \wedge Z_{i,j+1} \Longrightarrow \big(D_{i,k} = 1\big),$$
que é o que queríamos introduzir como bônus na função de custo $f$.
Então incluímos um novo parâmetro $\gamma > 0$ e atualizamos a função de custo como

$$f(Z) = \left\lVert L \right\rVert_\infty + \beta \, \left\lVert L-\overline{L}\right\rVert_1 - \gamma \sum_{i=0}^{N-1} \sum_{k=0}^{6} D_{i,k}.$$

ficando pendente, então, a inclusão de restrições na forma
$D_{i,k} \le \min \{Z_{i,j}, \; Z_{i,j+1} \}$.

<hr style="border-top: 1px solid gray; margin:.8em 0 1.2em">

## Restrições
Incluem-se, naturalmente, as restrições decorrentes do item $3$ acima.
Além destas, a restrição fundamental é atingir o mínimo de $M$ voluntários por turno:
$$\forall \, j: \; \sum_i Z_{ij} = M.$$

Além disso, gostaríamos de exigir que $Z \preceq A$, ou seja, que cada entrada $Z_{ij}$ seja igual ou inferior a $A_{ij}$, de modo a evitar a alocação de pessoas em turnos que não foram marcados como disponíveis: cada $(i, j)$ com $A_{ij} = 0$ impõe $Z_{ij} = 0$.

Contudo, tendo em vista que a planilha ainda está sendo preenchida, tal restrição foi incorporada à função de custo como penalização &mdash; fixamos um valor $\alpha$ de custo para cada entrada $(i,j)$ com $Z_{ij}=1$ e $A_{ij}=0$:
$$\alpha\sum_{i, j} \max(Z_{ij} - A_{ij}, \, 0) = \alpha\sum_{i, j} \left[ Z_{ij} - A_{ij} \right]_+.$$

Por fim, uma última restrição consiste em assegurar que ao menos um dos responsáveis/gestores da associação esteja presente em cada turno. Ou seja, filtrando a matriz $Z$ para algumas linhas específicas (correspondentes aos responsáveis &mdash; que sinalizamos no arquivo de dados com um "#" à esquerda do nome), pede-se que a soma de cada coluna da submatriz seja igual ou superior a $1$.

&nbsp;
## Soluções
&nbsp;

A seguir, chamamos a função `main()` do arquivo `schedule.py`, que monta, resolve e exibe a solução do problema para diversos valores de $M$, conforme a modelagem acima explícita.

In [5]:
def table_print(Z, names):
    for row, name in zip(Z, names):
        row = tuple(("x" if j else " ") for j in row)
        days = zip(row[0::2], row[1::2])  # pares diários (almoço & janta)
        days = " | ".join("{} {}".format(day, night) for day, night in days)
        print("[", days, "]:", name)

&nbsp;
### Caso M=5
&nbsp;

In [6]:
results = sc.main(table_fname, min_staff=5, verbose=0)
table_print(results["Z_array"], names)

Using license file /opt/gurobi901/gurobi.lic
Academic license - for non-commercial use only
[ x x |     |     |     |     |     |   x ]: Alex D. O.
[     |     |     |     |     |     |     ]: Andria S. O. R.
[     |     | x x |     | x   |     |     ]: Annatercia G. P.
[     |     |     |     | x x |   x |     ]: Beatriz S. B. S.
[ x x |     |   x |     |     |     |     ]: Bianca B. D.
[     |     |     |     |     |     |     ]: Breno Henrique O. S.
[     |     | x   |     | x x |     |     ]: Bryan P.
[     | x   |     | x   |     | x   |     ]: Carlos Eduardo S.
[   x | x x |     |     |     |     |     ]: Charlie V. S.
[     |     |     | x   |     |     | x x ]: Daisy S. D. Z. A.
[     |     |     | x   |     |     | x x ]: Danilo S. A. B.
[     |     |     |     |     |     |     ]: Fabiana P. P. V.
[     |     |     |     |     |     |     ]: Felipe O. M.
[     |     |     |     |   x | x x |     ]: Fred F. M.
[ x   |     |     |     |     |     | x   ]: Gabriel P. C.
[     | 

&nbsp;
### Caso M=6
&nbsp;

In [7]:
# Adicionando um limite de tempo (que nem deve ser necessário, mas enfim).
gurobi_options = {"verbose": True, "TimeLimit": 30}
results = sc.main(table_fname, min_staff=6, verbose=2, solver_options=gurobi_options)

# Programa de otimização linear com variáveis inteiras (MIP)

Dados de Entrada:

[Array: AVAILABLE]
[[1 1 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 1 1 0 0 1 1]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 1 1]
 [1 1 1 1 0 0 1 1 0 0 1 1 1 1]
 [1 1 0 0 1 1 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 1 1 1 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 

In [8]:
print("[Resultados para M=6]:")
table_print(results["Z_array"], names)

[Resultados para M=6]:
[ x x |     |     |     |     |     | x x ]: Alex D. O.
[     |     |     |     |     |     |     ]: Andria S. O. R.
[     |     | x x |     | x x |     |     ]: Annatercia G. P.
[     |     |     |     | x x | x x |     ]: Beatriz S. B. S.
[     |     |     | x x |     |     |   x ]: Bianca B. D.
[     |     |     |     |     |     |     ]: Breno Henrique O. S.
[     |     | x x |     |     |     | x x ]: Bryan P.
[     |     | x   | x   |     | x   |     ]: Carlos Eduardo S.
[     | x x |     |     |     |     | x x ]: Charlie V. S.
[     |     |     |   x |     |     | x x ]: Daisy S. D. Z. A.
[ x x |     |     |     |     | x x |     ]: Danilo S. A. B.
[     |     |     |     |     |     |     ]: Fabiana P. P. V.
[     |     |     |     |     |     |     ]: Felipe O. M.
[   x |     |   x |     |     | x x |     ]: Fred F. M.
[ x   |     |     |     |     |     | x   ]: Gabriel P. C.
[     | x x | x x |     |     |     |     ]: Hariom N. C.
[     |     |     |

&nbsp;
### Caso M=7
&nbsp;

In [9]:
results = sc.main(table_fname, min_staff=7, verbose=2, solver_options=gurobi_options)

# Programa de otimização linear com variáveis inteiras (MIP)

Dados de Entrada:

[Array: AVAILABLE]
[[1 1 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 1 1 0 0 1 1]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 1 1]
 [1 1 1 1 0 0 1 1 0 0 1 1 1 1]
 [1 1 0 0 1 1 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 1 1 1 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 

In [10]:
# print(*results.items(), sep="\n\n")
print("[Resultados para M=7]:")
table_print(results["Z_array"], names)

[Resultados para M=7]:
[ x x |     |     |     |     |     | x x ]: Alex D. O.
[     |     |     |     |     |     |     ]: Andria S. O. R.
[     |     | x x |     | x x |     |     ]: Annatercia G. P.
[ x x |     |     |     | x x |   x |     ]: Beatriz S. B. S.
[     | x x |     | x x |     |     |     ]: Bianca B. D.
[     |     |     |     |     |     |     ]: Breno Henrique O. S.
[     |     | x x |     |     |     | x x ]: Bryan P.
[     | x   | x   |     | x   | x   |     ]: Carlos Eduardo S.
[ x x | x x |     |     |     |     |     ]: Charlie V. S.
[ x   |     |     | x x |     | x x |     ]: Daisy S. D. Z. A.
[     |     |     | x x |     | x x |   x ]: Danilo S. A. B.
[     |     |     |     |     |     |     ]: Fabiana P. P. V.
[     |     |     |     |     |     |     ]: Felipe O. M.
[   x |     |     | x x |     |     | x x ]: Fred F. M.
[ x   |     |     |     |     |     | x   ]: Gabriel P. C.
[     | x x |     |     |     | x x |     ]: Hariom N. C.
[     |     |     |

&nbsp;
<header>
    <h3>To Be Continued...</h3>
</header>
&nbsp;